In [1]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
import BandGapPredictor as bgp
import numpy as np


e:\miniconda3\envs\AI\lib\site-packages\torch\distributed\_shard\partial_tensor.py:40: UserWarning: Please use DTensor instead and we are deprecating ShardedTensor.
  warnings.warn(DEPRECATE_MSG)
e:\miniconda3\envs\AI\lib\site-packages\torch\distributed\_shard\replicated_tensor.py:20: UserWarning: Please use DTensor instead and we are deprecating ShardedTensor.
  warnings.warn(DEPRECATE_MSG)


In [2]:
from torch.multiprocessing import Pool, Process, set_start_method
try:
     set_start_method('spawn')
except RuntimeError:
    pass
dev = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")
torch.set_float32_matmul_precision('high')
print(dev)

cuda


In [3]:

checkpoint_callback = ModelCheckpoint(
    dirpath='./checkpoints',
    filename='{epoch}-{val_loss:.2f}-{val_r2:.2f}',
    save_top_k=1,
    monitor='val_loss',
    every_n_epochs =1,
)
early_stop_callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0.01,
    patience=10,
    verbose=False,
    mode='min'
)

model = bgp.models.BandGap(lr=1e-3)

model = model.load_from_checkpoint(checkpoint_path="./checkpoints/epoch=19-val_loss=1.44-val_r2=-0.44.ckpt")

trainer = pl.Trainer(accelerator='gpu', devices=1, max_epochs=100,log_every_n_steps=1,callbacks=[checkpoint_callback])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [4]:
torch.cuda.empty_cache()

In [5]:
data  = bgp.dataloaders.LoadMolecules(molecules_root = r"E:\Datasets\BandGap\3d-boxels-molecule-for-bandgap-prediction\Data",band_root =r"E:\Datasets\BandGap\3d-boxels-molecule-for-bandgap-prediction" , max_samples=12500)
data = [x for x in data]
print(len(data))
data_module = bgp.dataloaders.InMemoryDataModule(data,32,4)


12500


In [7]:
trainer.fit(model,data_module)

e:\miniconda3\envs\AI\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:613: UserWarning: Checkpoint directory E:\Github\band-gap\checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type     | Params
-------------------------------------
0 | loss    | RMSELoss | 0     
1 | r2score | R2Score  | 0     
2 | model   | CNNModel | 9.6 M 
-------------------------------------
9.6 M     Trainable params
0         Non-trainable params
9.6 M     Total params
38.452    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

e:\miniconda3\envs\AI\lib\site-packages\pytorch_lightning\trainer\call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
import numpy as np
x , y = data_module.test_data[np.random.randint(0, len(data_module.test_data))]
x = torch.tensor(x).unsqueeze(0)
model.eval()
out = model(x)
print(out, y)
print(out-y)

In [ ]:
x , y = data_module.test_data[742]
# x to tensor
x = torch.tensor(x).unsqueeze(0).to(dev)
# model to eval
model.eval()
out = model(x)
print(out, y)

In [ ]:
e = trainer.test(model, data_module)

You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_loss_epoch        1.1162794828414917
      test_r2_epoch         -0.4330931007862091
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
